🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

## Creating custom components that incorporate logging

First, we set up structured logging using Python's built-in logging module. We will configure a logger for the component that includes structured information like timestamps, log level, and the name of the logger in each log entry.

In [1]:
import logging
import json

# Configure logging with a basic configuration that includes the level name and message.
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger("LoggingExampleComponent")

# Optionally, create a custom log handler to filter or modify logs (e.g., to mask sensitive data).
class SensitiveDataFilter(logging.Filter):
    def filter(self, record):
        if hasattr(record, 'msg') and "sensitive" in record.msg:
            record.msg = record.msg.replace("sensitive", "*****")
        return True

logger.addFilter(SensitiveDataFilter())


Step 2: Define the Custom Component with Structured Logging
This component will perform a simple task and log messages at different levels, demonstrating structured logging and the use of dynamic log levels.

In [2]:
from haystack import component

@component
class LoggingExampleComponent():

    @component.output_types(log_message=str, status=str)
    def run(self, input_data: str) -> dict:
        """
        Demonstrates logging within a custom component, with structured logging and sensitivity considerations.
        """
        try:
            # Log an info message with structured data
            logger.info(json.dumps({"event": "ProcessingStarted", "data_length": len(input_data)}))
            
            # Simulate processing and log a debug message
            processed_data = input_data.upper()
            logger.debug(json.dumps({"event": "ProcessingDebug", "message": f"Processed data to: {processed_data}"}))
            
            # Log a warning if input data is short
            if len(input_data) < 10:
                logger.warning(json.dumps({"event": "ShortInputData", "length": len(input_data)}))
            
            return {"log_message": f"Processed data: {processed_data}", "status": "success"}
        
        except Exception as e:
            # Log an error with structured data
            logger.error(json.dumps({"event": "ProcessingError", "error": str(e)}))
            return {"log_message": "An error occurred during processing.", "status": "error"}


Step 3: Dynamic Log Level Configuration
The verbosity of logs can be adjusted based on the environment (development or production) by setting the logger's level. This can be done at the start of the application or dynamically adjusted.

In [3]:
# For development, set to DEBUG to see all log messages.
logger.setLevel(logging.DEBUG)

# For production, you might want to set it to WARNING or ERROR to reduce verbosity.
# logger.setLevel(logging.WARNING)


Best Practices in Logging

* Consistent Logging Format: By using JSON format for structured logging, log messages are consistent and machine-readable, facilitating analysis and monitoring.
* Sensitive Information: The SensitiveDataFilter demonstrates how to filter out sensitive information from logs to protect privacy and comply with regulations.
* Dynamic Log Levels: Adjusting the logger's level based on the environment allows for more detailed logs during development and quieter logs in production, managing verbosity effectively.

In [4]:
if __name__ == "__main__":
    component = LoggingExampleComponent()
    result = component.run(input_data="Example data")
    print(result)


2025-10-28 13:10:16,772 - LoggingExampleComponent - INFO - {"event": "ProcessingStarted", "data_length": 12}
2025-10-28 13:10:16,773 - LoggingExampleComponent - DEBUG - {"event": "ProcessingDebug", "message": "Processed data to: EXAMPLE DATA"}


{'log_message': 'Processed data: EXAMPLE DATA', 'status': 'success'}


This `LoggingExampleComponent` showcases how to integrate advanced logging practices into Haystack custom components, enhancing the observability and maintainability of NLP pipelines.